In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_customers=pd.read_csv(r"C:\Users\lenovo\Downloads\ASSIGNMENT\python assignment 15_3_2025\customers.csv")
df_loans=pd.read_csv(r"C:\Users\lenovo\Downloads\ASSIGNMENT\python assignment 15_3_2025\loans.csv")
df_defaults=pd.read_csv(r"C:\Users\lenovo\Downloads\ASSIGNMENT\python assignment 15_3_2025\defaults.csv")
df_branch=pd.read_csv(r"C:\Users\lenovo\Downloads\ASSIGNMENT\python assignment 15_3_2025\branches.csv")
df_transactions=pd.read_csv(r"C:\Users\lenovo\Downloads\ASSIGNMENT\python assignment 15_3_2025\transactions.csv")
df_app=pd.read_csv(r"C:\Users\lenovo\Downloads\ASSIGNMENT\python assignment 15_3_2025\applications.csv")

In [ ]:
df=pd.merge([df_app,df_branch,df_customers,df_defaults,df_loans,df_transactions],axis=1)

In [ ]:
help(pd.merge)

In [ ]:
df_app.columns

In [ ]:
df_customers.columns

In [ ]:
df_app_cust=df_app.merge(df_customers, on ='Customer_ID')

In [ ]:
df_app_cust.columns

In [ ]:
df_app_cust_loan=df_app_cust.merge(df_loans, on ='Loan_ID')

In [ ]:
df_app_cust_loan.columns

In [ ]:
df_app_cust_loan_defaults=df_app_cust_loan.merge(df_defaults, on='Loan_ID')

In [ ]:
df_app_cust_loan_defaults.columns

In [ ]:
df_app_cust_loan_defaults_trans=df_app_cust_loan_defaults.merge(df_transactions,on='Customer_ID')

In [ ]:
df_app_cust_loan_defaults_trans.columns

In [ ]:
df_app_cust_loan_defaults_trans_branch=df_app_cust_loan_defaults_trans.merge(df_branch,on='Region')

In [ ]:
df_app_cust_loan_defaults_trans_branch.columns

In [ ]:
df_app_cust_loan_defaults_trans_branch.isna()

In [ ]:
df_app_cust_loan_defaults_trans_branch.isna().sum()

In [ ]:
df_app_cust_loan_defaults_trans_branch.shape

In [ ]:
df_app_cust_loan_defaults_trans_branch.drop('Rejection_Reason',axis=1,inplace=True)

In [ ]:
df=df_app_cust_loan_defaults_trans_branch.copy()

In [ ]:
df

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
df.head()

In [ ]:
for i in df.select_dtypes(['int64','float']):
    plt.figure(figsize=(3,3))
    sns.boxplot(y=i,data=df)
    plt.show()

In [ ]:
#outlier treatment
def remove_outlier(df,column):
    Q1=df[column].quantile(0.25)
    Q3=df[column].quantile(0.75)
    IQR=Q3-Q1
    lower_bound=Q1-(1.5*IQR)
    upper_bound=Q3+(1.5*IQR)
    return df[(df[column]>=lower_bound) & (df[column]<=upper_bound)]

In [ ]:
columns=df.select_dtypes(['int64','float']).columns

In [ ]:
columns

In [ ]:
# calling the function to remove outliers
df_clean=remove_outlier(df,columns)

In [ ]:
df_clean

In [ ]:
df.describe().T

In [ ]:
df1=df[['Loan_Amount','Credit_Score','EMI_Amount']]

In [ ]:
df1

In [ ]:
for i in df.select_dtypes(['int64','float64']):
    plt.figure(figsize=(3,3))
    sns.histplot(x=i,data=df)
                
    plt.show()

Regional trends in loan disbursement and defaults.

In [ ]:
#summarize the total loan regionwise
loan_by_region=df.groupby('Region')['Loan_Amount'].sum().reset_index()

In [ ]:
sns.barplot(x='Region',y='Loan_Amount',data=loan_by_region)

In [ ]:
df.columns

In [ ]:
df[['Default_Date', 'Default_Amount',
       'Default_Reason']]

In [ ]:
default_by_region=df.groupby('Region')['Default_Amount'].sum().reset_index()

In [ ]:
sns.barplot(x='Region',y='Default_Amount',data=default_by_region)

o Monthly trends in loan approvals and disbursements. 

In [ ]:
df.columns

In [ ]:
df[['Application_Date','Approval_Status','Loan_Disbursement_Amount']]

In [ ]:
df['Application_Date']=pd.to_datetime(df['Application_Date'])

In [ ]:
df['Application_Date'].info()

In [ ]:
Month = df['Application_Date'].dt.to_period('M')


In [ ]:
Month.count()

In [ ]:
monthly_approvals = df.groupby('Month').size()

In [ ]:
monthly_approvals

In [ ]:
df['Disbursal_Date'] = pd.to_datetime(df['Disbursal_Date'])

In [ ]:
Month = df['Disbursal_Date'].dt.to_period('M')

In [ ]:
monthly_disbursements = df.groupby('Month')['Loan_Disbursement_Amount'].sum().reset_index()

In [ ]:
monthly_disbursements

In [ ]:
sns.barplot(x='Month',y='Loan_Disbursement_Amount',data=monthly_disbursements)

3. Default Risk Analysis 
● Correlation Between Loan Attributes and Defaults: 
o Calculate correlations between Loan_Amount, Interest_Rate, 
Credit_Score, and Default_Flag (a binary indicator for default). 

In [ ]:
df['Default_Flag'] = np.where(df['Loan_Status'] == 'Defaulted', 1, 0)

In [ ]:
correlation_data = df[['Loan_Amount', 'Interest_Rate', 'Credit_Score', 'Default_Flag']].corr()

In [ ]:
correlation_data

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_data, annot=True, cmap='coolwarm', square=True)
plt.title('correlation_data')
plt.show()

● Pairwise Correlation Analysis: 
o Create a heatmap to visualize the correlations between key 
variables, such as EMI_Amount, Overdue_Amount, and 
Default_Amount. 

In [ ]:
correlation_data1=df[['EMI_Amount','Overdue_Amount','Default_Amount']]
correlation_data1

In [ ]:
correlation=correlation_data1.corr()
correlation

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(correlation, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Pairwise Correlation Analysis")
plt.show()


● Correlation Between Branch Metrics and Defaults: 
o Analyze the relationship between branch performance metrics (e.g., 
Delinquent_Loans, Loan_Disbursement_Amount) and default rates. 

In [ ]:
df.columns

In [ ]:
df[['Total_Active_Loans','Delinquent_Loans','Loan_Disbursement_Amount']]

In [ ]:
# Step 2: Count defaults per branch
branch_defaults = df.groupby("Branch_ID")["Loan_ID_x"].count().reset_index()
branch_defaults.columns = ["Branch_ID", "Default_Count"]

In [ ]:
branch_defaults.columns

In [ ]:
branch_data = df.merge(branch_defaults, on="Branch_ID", how="left").fillna(0)

In [ ]:
branch_data

In [ ]:
branch_data["Default_Rate"] = (branch_data["Default_Count"] / branch_data["Total_Active_Loans"]) * 100

In [ ]:
branch_data["Default_Rate"]

In [ ]:
correlation_matrix = branch_data[[
    "Total_Active_Loans", 
    "Delinquent_Loans", 
    "Loan_Disbursement_Amount", 
    "Default_Rate"
]].corr()

correlation_matrix

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix: Branch Metrics vs Defaults")
plt.show()

4. Branch and Regional Performance 
● Rank branches by: 
o Loan disbursement volume. 
o Processing time efficiency. 
o Default rates and recovery rates. 
● Compare branch performance across regions. 

In [ ]:
branch_defaults = df.groupby("Branch_ID")["Loan_Disbursement_Amount"].sum().reset_index()
branch_defaults

In [ ]:
branch_disbursement = branch_defaults.sort_values(by='Loan_Disbursement_Amount', ascending=False)
branch_disbursement

In [ ]:
branch_disbursement.head(10)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))

# Column names should be in quotes
sns.barplot(x='Branch_ID', y='Loan_Disbursement_Amount', data=branch_disbursement.head(10))
plt.title('Top 10 Branches')
plt.xticks(rotation=90)  
plt.show()


In [ ]:
df['Application_Date'] = pd.to_datetime(df['Application_Date'])
df['Approval_Date'] = pd.to_datetime(df['Approval_Date'])

In [ ]:
#calculate proccesing time
df['proccesing_time']=(df['Approval_Date']-df['Application_Date'])
df['proccesing_time']

In [ ]:
Branch_efficiency=df.groupby('Branch_ID')['proccesing_time'].mean().reset_index()
Branch_efficiency.head(10)

In [ ]:
#rank
Branch_efficiency_sorted=Branch_efficiency.sort_values(by='proccesing_time')
Branch_efficiency_sorted

In [ ]:
Branch_efficiency_sorted.head(10)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x='Branch_ID',y='proccesing_time',data=Branch_efficiency_sorted.head(10))
plt.title('top 10 sortrd branch')
plt.xticks(rotation=90)
plt.show()

In [ ]:
#calculate total loan
Branch_total_loans=df.groupby('Branch_ID')['Loan_ID_y'].count().reset_index(name='Total_Loans')
Branch_total_loans

In [ ]:
#calculate default loan
branch_defaults = df[df['Default_Amount'].notna()].groupby('Branch_ID')['Loan_ID_y'].count().reset_index(name='Defaulted_Loans')
branch_defaults

In [ ]:
# merge both table defaulted loan & total loan
branch_performance = pd.merge(Branch_total_loans, branch_defaults, on='Branch_ID', how='left').fillna(0)
branch_performance

In [ ]:
# Calculate Default Rate
branch_performance['Default_Rate (%)'] = (branch_performance['Defaulted_Loans'] / branch_performance['Total_Loans']) * 100
branch_performance['Default_Rate (%)'].head(10)

In [ ]:
#calculate recury rate
recovery_data = df.groupby('Branch_ID').agg({'Default_Amount': 'sum', 'Recovery_Amount': 'sum'}).reset_index()
recovery_data['Recovery_Rate (%)'] = (recovery_data['Recovery_Amount'] / recovery_data['Default_Amount']) * 100

In [ ]:
recovery_data['Recovery_Rate (%)'].head(10)

In [ ]:
final_performance = pd.merge(branch_performance, recovery_data[['Branch_ID', 'Recovery_Rate (%)']], on='Branch_ID', how='left').fillna(0)

In [ ]:
final_performance[['Branch_ID', 'Default_Rate (%)', 'Recovery_Rate (%)']].head(10)

In [ ]:
plt.figure(figsize=(12, 10))
sns.barplot(x='Branch_ID', y='Default_Rate (%)', data=final_performance.sort_values('Default_Rate (%)', ascending=False).head(10))
plt.title('Top 10 Branches with Highest Default Rates')
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
sns.barplot(x='Branch_ID', y='Recovery_Rate (%)', data=final_performance.sort_values('Recovery_Rate (%)', ascending=False).head(10))
plt.title('Top 10 Branches with Highest Recovery Rates')
plt.xticks(rotation=90)
plt.show()

Compare branch performance across regions.

5. Customer Segmentation 
● Segment customers by income, credit score, and loan status. 
● Identify high-risk and high-value customer groups. 
● Analyze repayment behavior across segments.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
features = df[['Annual_Income', 'Credit_Score', 'Default_Flag']].fillna(0)
features

In [ ]:

# Standardizing data for K-Means
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [ ]:
# K-Means Clustering (Choosing 3 clusters for segmentation)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df['Segment'] = kmeans.fit_predict(scaled_features)
df['Segment']

In [ ]:
# Vi)sualizing Segmentation: Income vs Credit Score
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Annual_Income', y='Credit_Score', hue='Segment', data=df, palette='Set2')
plt.title('Customer Segmentation: Income vs Credit Score')
plt.show()

In [ ]:
# Visualizing Loan Status by Segments
plt.figure(figsize=(10, 6))
sns.countplot(x='Segment', hue='Default_Flag', data=df, palette='coolwarm')
plt.title('Loan Status Distribution Across Segments')
plt.show()

 Identify high-risk and high-value customer groups.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
#  Select relevant features
features = df[['Annual_Income', 'Credit_Score', 'Loan_Amount']]
features

In [ ]:
#  Data Scaling
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)  
df['Cluster'] = kmeans.fit_predict(scaled_features)
df['Cluster']

In [ ]:
#Visualize the clusters
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Annual_Income', y='Credit_Score', hue='Cluster', data=df, palette='viridis')
plt.title('Customer Segmentation: Income vs Credit Score')
plt.show()

In [ ]:
cluster_summary = df.groupby('Cluster').agg({
    'Annual_Income': 'mean',
    'Credit_Score': 'mean',
    'Loan_Amount': 'mean',
    'Default_Flag': 'mean'
})

print(cluster_summary)

● Analyze repayment behavior across segments. 

In [ ]:
df['On_Time_Payment'] = df['Payment_Type'].apply(lambda x: 1 if x == 'EMI Payment' else 0)

In [ ]:
df['On_Time_Payment']

In [ ]:
repayment_behavior = df.groupby('Customer_ID').agg({
    'On_Time_Payment': 'sum',
    'Amount': 'sum',
    'Default_Amount': 'sum',
    'Annual_Income': 'first',
    'Credit_Score': 'first'
}).reset_index()

In [ ]:
#Segment analysis
# Creating segments based on income and credit score
def assign_segment(row):
    if row['Credit_Score'] > 750 and row['Annual_Income'] > 1000000:
        return 'High Value'
    elif row['Credit_Score'] < 600:
        return 'High Risk'
    else:
        return 'Moderate Risk'

repayment_behavior['Segment'] = repayment_behavior.apply(assign_segment, axis=1)


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='Segment', y='On_Time_Payment', data=repayment_behavior)
plt.title('Repayment Behavior Across Customer Segments')
plt.show()

In [ ]:
#  Summary Analysis
segment_summary = repayment_behavior.groupby('Segment').agg({
    'On_Time_Payment': 'mean',
    'Amount': 'mean',
    'Default_Amount': 'mean'
}).round(2)

print(segment_summary)

6. Advanced Statistical Analysis 
. Correlation Analysis for Default Risks: 
o Examine the correlation between Credit_Score, Loan_Amount, 
Interest_Rate, Overdue_Amount, and Default_Flag.

In [ ]:
# Select relevant columns
corr_df = df[['Credit_Score', 'Loan_Amount', 'Interest_Rate', 'Overdue_Amount', 'Default_Flag']]

# Calculate correlation matrix
corr_matrix = corr_df.corr()

# Visualize correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', square=True)
plt.title('Correlation Matrix for Default Risks')
plt.show()

In [ ]:
# Calculate pairwise correlations
df['Recovery_Rate'] = df['Recovery_Amount'] /['Default_Amount'] * 100
corr_matrix = df[['EMI_Amount','recovery_rate', 'Loan_Amount']].corr()

# Generate heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', square=True)
plt.title('Pairwise Correlation Heatmap')
plt.show()

3. Branch-Level Correlation: 
o Explore the relationship between branch performance metrics 
(Delinquent_Loans, Loan_Disbursement_Amount, Recovery_Rate) 
and overall efficiency. 

In [ ]:
# Calculate overall efficiency
branch_data['Efficiency_Ratio'] = (df['Loan_Disbursement_Amount'] /['Delinquent_Loans']) * df['Default_Amount']

# Analyze correlations
corr_matrix = branch_data[['Delinquent_Loans', 'Loan_Disbursement_Amount', 'Default_Amount', 'Efficiency_Ratio']].corr()

# Visualize relationships
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', square=True)
plt.title('Branch-Level Correlations')
plt.show()

7. Transaction and Recovery Analysis 
● Analyze penalty payments and overdue trends. 
● Evaluate recovery rates by Default_Reason and Legal_Action. 
● Compare recovery rates across regions and branches

In [ ]:
 #Calculate penalty payments
penalty_payments = df[df['Overdue_Fee'] > 0]
total_penalty_amount = penalty_payments['Overdue_Fee'].sum()

# Analyze overdue trends
overdue_amounts = df[df['Overdue_Fee'] > 0]
overdue_distribution = overdue_amounts['Overdue_Fee'].value_counts()

# Visualize the analysis
plt.figure(figsize=(10, 6))
plt.bar(overdue_distribution.index, overdue_distribution.values)
plt.xlabel('Overdue Fee')
plt.ylabel('Frequency')
plt.title('Overdue Fee Distribution')
plt.show()

8. EMI Analysis 

● Analyze the relationship between EMI amounts and default probabilities. 
● Identify thresholds for EMI amounts where defaults are most likely. 
● Compare EMI trends across loan types.

● Analyze the relationship between EMI amounts and default probabilities. 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1: Group data to calculate default rates for different EMI ranges
df['EMI_Range'] = pd.cut(df['EMI_Amount'], bins=[0, 5000, 10000, 20000, 50000, 100000], labels=['0-5K', '5K-10K', '10K-20K', '20K-50K', '50K-100K'])

# Step 2: Calculate default rates
emi_analysis = df.groupby('EMI_Range').agg(
    Total_Loans=('Loan_ID_x', 'count'),
    Defaults=('Default_Flag', 'sum')
).reset_index()

# Step 3: Calculate the probability of default
emi_analysis['Default_Rate'] = (emi_analysis['Defaults'] / emi_analysis['Total_Loans']) * 100

# Step 4: Visualize the EMI Analysis
plt.figure(figsize=(10, 6))
sns.barplot(x='EMI_Range', y='Default_Rate', data=emi_analysis, palette='coolwarm')
plt.title('Default Probability by EMI Range')
plt.xlabel('EMI Range')
plt.ylabel('Default Probability (%)')
plt.show()


● Identify thresholds for EMI amounts where defaults are most likely. 

In [ ]:
df_sorted = df[['EMI_Amount', 'Default_Flag']].sort_values('EMI_Amount')

#: Create EMI bins with smaller intervals for detailed analysis
df_sorted['EMI_Bins'] = pd.cut(df_sorted['EMI_Amount'], bins=range(0, 100000, 5000))
df_sorted
df_sorted['EMI_Bins']

# : Calculate default rates for each EMI bin with observed=False
emi_threshold_analysis = df_sorted.groupby('EMI_Bins', observed=False).agg(
    Total_Loans=('EMI_Amount', 'count'),
    Defaults=('Default_Flag', 'sum')
).reset_index()


# : Calculate default rates
emi_threshold_analysis['Default_Rate'] = (emi_threshold_analysis['Defaults'] / 
                                          emi_threshold_analysis['Total_Loans']) * 100
emi_threshold_analysis['Default_Rate']

In [ ]:
df_sorted.head(10)

 Compare EMI trends across loan types

In [ ]:
emi_trends = df.groupby('Loan_Purpose').agg(
    Avg_EMI=('EMI_Amount', 'mean'),
    Median_EMI=('EMI_Amount', 'median'),
    Default_Rate=('Default_Flag', 'mean')
).reset_index()

In [ ]:
plt.figure(figsize=(12, 6))

# Bar plot for Average EMI by Loan Type
sns.barplot(x='Loan_Purpose', y='Avg_EMI', data=emi_trends, palette='Blues')
plt.title('Average EMI Amount by Loan Type')
plt.xticks(rotation=45)
plt.show()

# Bar plot for Default Rate by Loan Type
plt.figure(figsize=(12, 6))
sns.barplot(x='Loan_Purpose', y='Default_Rate', data=emi_trends, palette='Reds')
plt.title('Default Rate by Loan Type')
plt.xticks(rotation=45)
plt.show()


9. Loan Application Insights 
● Calculate approval and rejection rates for loan applications. 

In [ ]:
# Calculate total applications
total_applications = len(df)

# Calculate approved and rejected applications
approved_count = (df['Approval_Status'] == 'Approved').sum()
rejected_count = (df['Approval_Status'] == 'Rejected').sum()

# Calculate rates
approval_rate = (approved_count / total_applications) * 100
rejection_rate = (rejected_count / total_applications) * 100

print(f"Total Applications: {total_applications}")
print(f"Approval Rate: {approval_rate:.2f}%")
print(f"Rejection Rate: {rejection_rate:.2f}%")


In [ ]:
labels = ['Approved', 'Rejected']
sizes = [approved_count, rejected_count]
colors = ['#4CAF50', '#F44336']

plt.figure(figsize=(6, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%.2f%%', startangle=140, wedgeprops={'edgecolor': 'black'})
plt.title('Loan Application Approval vs Rejection Rates')
plt.show()

 Identify the most common reasons for loan rejection.

In [ ]:
# Filter rejected applications
rejected_loans = df[df['Approval_Status'] == 'Rejected']

# Possible factors linked to rejection
rejection_factors = rejected_loans[['Credit_Score', 'Annual_Income', 
                                    'Employment_Status', 'Processing_Fee', 
                                    'Loan_Amount', 'Collateral_Details']].describe()


# Add a column to explain possible rejection reasons
rejected_loans['Potential_Rejection_Reason'] = np.where(rejected_loans['Credit_Score'] < 600, 
                                                        'Low Credit Score',
                                               np.where(rejected_loans['Annual_Income'] < 200000, 
                                                        'Low Income',
                                               np.where(rejected_loans['Employment_Status'] == 'Unemployed', 
                                                        'Unstable Employment',
                                                        'Other Factors')))



In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
sns.countplot(x='Potential_Rejection_Reason', data=rejected_loans, order=rejected_loans['Potential_Rejection_Reason'].value_counts().index)
plt.title('Common Rejection Reasons')
plt.xticks(rotation=45)
plt.show()


● Compare application processing fees between approved and rejected 
applications. 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Group by Approval Status and calculate average processing fees
fee_comparison = df.groupby('Approval_Status')['Processing_Fee'].agg(['mean', 'median', 'max', 'min']).reset_index()
print(fee_comparison)


In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(x='Approval_Status', y='Processing_Fee', data=df, palette='coolwarm')
plt.title('Comparison of Processing Fees: Approved vs. Rejected Applications')
plt.show()


10. Recovery Effectiveness 
● Determine the effectiveness of recovery efforts by calculating the ratio of 
Recovery_Amount to Default_Amount. 

In [ ]:
# Calculate Recovery Rate
df['Recovery_Rate'] = df['Recovery_Amount'] / df['Default_Amount']

# Display key metrics
recovery_summary = df[['Default_Amount', 'Recovery_Amount', 'Recovery_Rate']].describe()
recovery_summary

plt.figure(figsize=(8, 5))
sns.histplot(df['Recovery_Rate'], bins=20, kde=True, color='skyblue')
plt.title('Distribution of Recovery Rates')
plt.xlabel('Recovery Rate')
plt.ylabel('Frequency')
plt.show()


In [ ]:
recovery_by_reason = df.groupby('Default_Reason')['Recovery_Rate'].mean().reset_index()

plt.figure(figsize=(10, 5))
sns.barplot(y='Recovery_Rate', x='Default_Reason', data=recovery_by_reason, palette='viridis')
plt.title('Average Recovery Rate by Default Reason')
plt.xlabel('Average Recovery Rate')
plt.show()



 Compare recovery rates for defaults with and without legal actions.

In [ ]:
# Calculate Recovery Rate
df['Recovery_Rate'] = df['Recovery_Amount'] / df['Default_Amount']

# Group by 'Legal_Action' and calculate mean recovery rates
recovery_comparison = df.groupby('Legal_Action')['Recovery_Rate'].mean().reset_index()

recovery_comparison

plt.figure(figsize=(8, 5))
sns.barplot(x='Legal_Action', y='Recovery_Rate', data=recovery_comparison, palette='coolwarm')
plt.title('Comparison of Recovery Rates: With vs Without Legal Actions')
plt.xlabel('Legal Action Status')
plt.ylabel('Average Recovery Rate')
plt.show()



 Analyze branch-wise recovery performance. 

In [ ]:
# Calculate Recovery Rate
df['Recovery_Rate'] = df['Recovery_Amount'] / df['Default_Amount']

# Group by Branch_ID and calculate the average recovery rate
branch_recovery = df.groupby('Branch_ID')['Recovery_Rate'].mean().reset_index()

# Sort by Recovery Rate in descending order
branch_recovery_sorted = branch_recovery.sort_values(by='Recovery_Rate', ascending=False)

print(branch_recovery_sorted.head(10))  # Display top 10 branches with highest recovery rates

plt.figure(figsize=(12, 6))
sns.barplot(x='Branch_ID', y='Recovery_Rate', data=branch_recovery_sorted.head(10), palette='viridis')
plt.title('Top 10 Branches by Recovery Performance')
plt.xlabel('Branch ID')
plt.ylabel('Average Recovery Rate')
plt.xticks(rotation=45)
plt.show()



11. Loan Disbursement Efficiency 

 Analyze the time from application to loan disbursement and identify 
bottlenecks. 

In [ ]:
# Convert date columns to datetime format
df['Application_Date'] = pd.to_datetime(df['Application_Date'])
df['Disbursal_Date'] = pd.to_datetime(df['Disbursal_Date'])

# Calculate Processing Time (in days)
df['Processing_Time'] = (df['Disbursal_Date'] - df['Application_Date']).dt.days

# Display the first few rows to verify
print(df[['Application_ID', 'Application_Date', 'Disbursal_Date', 'Processing_Time']].head())

# Calculate average processing time for each branch
branch_processing_time = df.groupby('Branch_ID')['Processing_Time'].mean().reset_index()

# Sort by highest processing time
branch_processing_sorted = branch_processing_time.sort_values(by='Processing_Time', ascending=False)

print(branch_processing_sorted.head(10))  # Display top 10 slowest branches

plt.figure(figsize=(12, 6))
sns.barplot(x='Branch_ID', y='Processing_Time', data=branch_processing_sorted.head(10), palette='Reds')
plt.title('Top 10 Branches with Longest Processing Times')
plt.xlabel('Branch ID')
plt.ylabel('Average Processing Time (Days)')
plt.xticks(rotation=45)
plt.show()



 Compare average processing times across branches.

In [ ]:
# Convert date columns to datetime format
df['Application_Date'] = pd.to_datetime(df['Application_Date'])
df['Disbursal_Date'] = pd.to_datetime(df['Disbursal_Date'])

# Calculate Processing Time (in days)
df['Processing_Time'] = (df['Disbursal_Date'] - df['Application_Date']).dt.days

# Calculate average processing time for each branch
branch_processing_time = df.groupby('Branch_ID')['Processing_Time'].mean().reset_index()

# Sort the data to view highest and lowest processing times
branch_processing_time_sorted = branch_processing_time.sort_values(by='Processing_Time', ascending=False)

branch_processing_time_sorted

plt.figure(figsize=(12, 6))
sns.barplot(x='Branch_ID', y='Processing_Time', data=branch_processing_time_sorted, palette='viridis')
plt.title('Average Processing Time Across Branches')
plt.xlabel('Branch ID')
plt.ylabel('Average Processing Time (Days)')
plt.xticks(rotation=45)
plt.show()



12. Profitability Analysis 

● Calculate the total interest income generated across all loans. 

In [ ]:
# Ensure data types are correct
df['Loan_Amount'] = pd.to_numeric(df['Loan_Amount'], errors='coerce')
df['Interest_Rate'] = pd.to_numeric(df['Interest_Rate'], errors='coerce')
df['Loan_Term'] = pd.to_numeric(df['Loan_Term'], errors='coerce')

# Calculate Interest Income for each loan
df['Interest_Income'] = df['Loan_Amount'] * (df['Interest_Rate'] / 100) * (df['Loan_Term'] / 12)

# Calculate total interest income
total_interest_income = df['Interest_Income'].sum()

print(f"Total Interest Income Generated: ₹{total_interest_income:,.2f}")


● Identify the most profitable loan purposes based on interest earnings. 

In [ ]:
# Step 1: Calculate Interest Income
df['Interest_Income'] = (df['Loan_Amount'] * df['Interest_Rate'] * df['Loan_Term']) / 12

# Step 2: Aggregate by Loan Purpose
profit_by_purpose = df.groupby('Loan_Purpose')['Interest_Income'].sum().reset_index()

# Step 3: Sort and Display Top Profitable Loan Purposes
top_profitable_purposes = profit_by_purpose.sort_values(by='Interest_Income', ascending=False)

# Display Top 5
print(top_profitable_purposes.head(5))

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.barplot(y='Interest_Income', x='Loan_Purpose', data=top_profitable_purposes.head(10), palette='viridis')
plt.title('Top 10 Most Profitable Loan Purposes')
plt.xlabel('Total Interest Income')
plt.ylabel('Loan Purpose')
plt.show()


● Compare profitability metrics for branches across regions.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Calculate Profitability (Interest Income)
df['Interest_Income'] = (df['Loan_Amount'] * df['Interest_Rate'] * df['Loan_Term']) / 12

# Step 2: Aggregate Profitability by Branch and Region
profit_by_branch = df.groupby(['Branch_ID', 'Region'])['Interest_Income'].sum().reset_index()

# Step 3: Visualizing Profitability Across Branches
plt.figure(figsize=(12, 6))
sns.barplot(x='Branch_ID', y='Interest_Income', hue='Region', data=profit_by_branch)
plt.title('Branch-Wise Profitability Across Regions')
plt.xlabel('Branch ID')
plt.ylabel('Total Interest Income')
plt.legend(title='Region')
plt.xticks(rotation=45)
plt.show()

# Step 4: Regional Summary (Optional)
profit_by_region = profit_by_branch.groupby('Region')['Interest_Income'].sum().reset_index()

plt.figure(figsize=(8, 5))
sns.barplot(x='Region', y='Interest_Income', data=profit_by_region, palette='coolwarm')
plt.title('Total Profitability by Region')
plt.xlabel('Region')
plt.ylabel('Total Interest Income')
plt.show()


13. Geospatial Analysis

● Map the distribution of active loans across regions. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Filter active loans
active_loans = df[df['Loan_Status'] == 'Active']

# Step 2: Group by Region and count active loans
active_loans_by_region = active_loans.groupby('Region')['Loan_ID_x'].count().reset_index()
active_loans_by_region.rename(columns={'Loan_ID_x': 'Active_Loan_Count'}, inplace=True)

# Step 3: Visualizing Active Loans Distribution
plt.figure(figsize=(10, 6))
sns.barplot(x='Region', y='Active_Loan_Count', data=active_loans_by_region, palette='Blues_d')
plt.title('Distribution of Active Loans Across Regions')
plt.xlabel('Region')
plt.ylabel('Number of Active Loans')
plt.show()


● Compare default rates across different geographic regions. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Data Preparation
default_data = df[['Region', 'Loan_ID_x', 'Default_Flag']]

# Step 2: Calculate total loans and defaulted loans by region
region_defaults = default_data.groupby('Region').agg(
    Total_Loans=('Loan_ID_x', 'count'),
    Defaulted_Loans=('Default_Flag', 'sum')
).reset_index()

# Step 3: Calculate default rate
region_defaults['Default_Rate (%)'] = (region_defaults['Defaulted_Loans'] / 
                                       region_defaults['Total_Loans']) * 100

# Step 4: Visualize the results
plt.figure(figsize=(10, 6))
sns.barplot(x='Region', y='Default_Rate (%)', data=region_defaults, palette='Reds')
plt.title('Default Rates Across Geographic Regions')
plt.xlabel('Region')
plt.ylabel('Default Rate (%)')
plt.show()

# Display the table for reference
print(region_defaults)


● Visualize the loan disbursement trends for rural vs. urban areas. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Data Preparation
df['Disbursal_Date'] = pd.to_datetime(df['Disbursal_Date'])  # Ensure date format is correct

# Step 2: Filter and Group Data
rural_data = df[df['Region'] == 'Rural'].groupby(pd.Grouper(key='Disbursal_Date', freq='M'))['Loan_Disbursement_Amount'].sum()
urban_data = df[df['Region'] == 'Urban'].groupby(pd.Grouper(key='Disbursal_Date', freq='M'))['Loan_Disbursement_Amount'].sum()

# Step 3: Visualization
plt.figure(figsize=(12, 6))
plt.plot(rural_data.index, rural_data, label='Rural', marker='o')
plt.plot(urban_data.index, urban_data, label='Urban', marker='o')
plt.title('Loan Disbursement Trends: Rural vs. Urban')
plt.xlabel('Time (Monthly)')
plt.ylabel('Loan Disbursement Amount')
plt.legend()
plt.grid(True)
plt.show()


14. Default Trends 

 Analyze the number of defaults over time to identify patterns. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Ensure the Default_Date column is in proper datetime format
df['Default_Date'] = pd.to_datetime(df['Default_Date'], errors='coerce')

# Group by month to track trends
monthly_defaults = df[df['Default_Flag'] == 1].groupby(pd.Grouper(key='Default_Date', freq='M')).size()

# Group by year if a broader view is required
yearly_defaults = df[df['Default_Flag'] == 1].groupby(pd.Grouper(key='Default_Date', freq='Y')).size()

In [ ]:
plt.figure(figsize=(12, 6))

# Monthly Defaults Plot
plt.plot(monthly_defaults.index, monthly_defaults, marker='o', label='Monthly Defaults', color='red')

# Yearly Defaults Plot (Optional)
plt.plot(yearly_defaults.index, yearly_defaults, marker='o', linestyle='--', label='Yearly Defaults', color='blue')

plt.title('Default Trends Over Time')
plt.xlabel('Time Period')
plt.ylabel('Number of Defaults')
plt.legend()
plt.grid(True)
plt.show()


● Calculate the average default amount for different loan purposes. 

In [ ]:
import pandas as pd

# Grouping by 'Loan_Purpose' and calculating the average default amount
avg_default_by_purpose = df.groupby('Loan_Purpose')['Default_Amount'].mean().reset_index()

# Sorting by highest average default amount
avg_default_by_purpose_sorted = avg_default_by_purpose.sort_values(by='Default_Amount', ascending=False)

# Display the top 10 loan purposes with the highest average default amounts
print(avg_default_by_purpose_sorted.head(10))

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.barplot(y='Default_Amount', x='Loan_Purpose', data=avg_default_by_purpose_sorted.head(10), palette='Reds')
plt.title('Top 10 Loan Purposes with Highest Average Default Amounts')
plt.xlabel('Average Default Amount')
plt.ylabel('Loan Purpose')
plt.show()



● Compare default rates across customer income categories. 

In [ ]:
import pandas as pd

# Creating income categories
income_bins = [0, 30000, 60000, 100000, 200000, 500000, float('inf')]
income_labels = ['<30K', '30K-60K', '60K-100K', '100K-200K', '200K-500K', '500K+']

# Adding an 'Income_Category' column
df['Income_Category'] = pd.cut(df['Annual_Income'], bins=income_bins, labels=income_labels)

# Calculating default rates for each income category
default_rates_by_income = df.groupby('Income_Category')['Default_Flag'].mean().reset_index()

print(default_rates_by_income)

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.barplot(x='Income_Category', y='Default_Flag', data=default_rates_by_income, palette='Blues')
plt.title('Default Rates Across Income Categories')
plt.xlabel('Income Category')
plt.ylabel('Default Rate')
plt.show()



15. Branch Efficiency 

● Calculate the average loan disbursement time for each branch. 

In [ ]:
import pandas as pd

# Ensure date columns are in datetime format
df['Application_Date'] = pd.to_datetime(df['Application_Date'])
df['Disbursal_Date'] = pd.to_datetime(df['Disbursal_Date'])

# Calculate disbursement time in days
df['Disbursement_Time'] = (df['Disbursal_Date'] - df['Application_Date']).dt.days

# Calculate average disbursement time for each branch
avg_disbursement_time = df.groupby('Branch_ID')['Disbursement_Time'].mean().reset_index()

avg_disbursement_time


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.barplot(x='Branch_ID', y='Disbursement_Time', data=avg_disbursement_time.sort_values('Disbursement_Time'), palette='coolwarm')
plt.title('Average Loan Disbursement Time by Branch')
plt.xlabel('Branch ID')
plt.ylabel('Average Disbursement Time (Days)')
plt.xticks(rotation=45)
plt.show()


● Identify branches with the highest number of rejected applications.

In [ ]:
# Filter rejected applications
rejected_apps = df[df['Approval_Status'] == 'Rejected']

# Count the number of rejections for each branch
branch_rejections = rejected_apps.groupby('Branch_ID').size().reset_index(name='Rejection_Count')

# Sort branches by rejection count (highest first)
top_rejected_branches = branch_rejections.sort_values('Rejection_Count', ascending=False).head(10)

top_rejected_branches

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.barplot(x='Branch_ID', y='Rejection_Count', data=top_rejected_branches, palette='Reds')
plt.title('Top 10 Branches with Highest Rejected Applications')
plt.xlabel('Branch ID')
plt.ylabel('Number of Rejected Applications')
plt.xticks(rotation=45)
plt.show()



● Compare branch efficiency based on customer satisfaction metrics (if 
available).

In [ ]:
# Calculate branch efficiency metrics
branch_efficiency = df.groupby('Branch_ID').agg({
    'Avg_Processing_Time': 'mean',
    'Loan_Disbursement_Amount': 'sum',
    'Delinquent_Loans': 'sum',
    'Recovery_Amount': 'sum',
    'Default_Amount': 'sum'
}).reset_index()

# Add recovery rate column
branch_efficiency['Recovery_Rate'] = branch_efficiency['Recovery_Amount'] / branch_efficiency['Default_Amount']

# Sort by Avg_Processing_Time to identify fastest branches
fastest_branches = branch_efficiency.sort_values('Avg_Processing_Time').head(10)

fastest_branches

plt.figure(figsize=(10, 6))
sns.barplot(x='Branch_ID', y='Avg_Processing_Time', data=fastest_branches, palette='Blues')
plt.title('Top 10 Branches by Processing Efficiency')
plt.xlabel('Branch ID')
plt.ylabel('Average Processing Time (Days)')
plt.xticks(rotation=45)
plt.show()


In [ ]:
top_recovery_branches = branch_efficiency.sort_values('Recovery_Rate', ascending=False).head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x='Branch_ID', y='Recovery_Rate', data=top_recovery_branches, palette='Greens')
plt.title('Top 10 Branches by Recovery Rates')
plt.xlabel('Branch ID')
plt.ylabel('Recovery Rate')
plt.xticks(rotation=45)
plt.show()


16. Time-Series Analysis 

● Analyze monthly loan disbursement trends over the last 5 years. 

In [ ]:
print(df[['Disbursal_Date', 'Loan_Disbursement_Amount']].dtypes)

df['Disbursal_Date'] = pd.to_datetime(df['Disbursal_Date'], errors='coerce')

df['Loan_Disbursement_Amount'] = pd.to_numeric(df['Loan_Disbursement_Amount'], errors='coerce')

last_5_years = df[df['Disbursal_Date'] >= pd.to_datetime('today') - pd.DateOffset(years=5)]

last_5_years['Year_Month'] = last_5_years['Disbursal_Date'].dt.to_period('M')

monthly_disbursement = last_5_years.groupby('Year_Month')['Loan_Disbursement_Amount'].sum().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(x='Year_Month', y='Loan_Disbursement_Amount', data=monthly_disbursement, marker='o', color='blue')
plt.title('Monthly Loan Disbursement Trends (Last 5 Years)')
plt.xlabel('Year-Month')
plt.ylabel('Loan Disbursement Amount')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

● Compare monthly default rates across regions. 

In [ ]:
# Convert 'Disbursal_Date' to datetime
df['Disbursal_Date'] = pd.to_datetime(df['Disbursal_Date'], errors='coerce')

# Filter valid data (optional but useful)
df = df.dropna(subset=['Disbursal_Date', 'Region', 'Default_Flag'])

# Create 'Year_Month' column for grouping
df['Year_Month'] = df['Disbursal_Date'].dt.to_period('M')


In [ ]:
# Group by Year_Month and Region
monthly_defaults = df.groupby(['Year_Month', 'Region']).agg(
    Total_Loans=('Loan_ID_x', 'count'),
    Defaults=('Default_Flag', 'sum')
).reset_index()

# Calculate Default Rate
monthly_defaults['Default_Rate'] = (monthly_defaults['Defaults'] / 
                                    monthly_defaults['Total_Loans']) * 100


In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='Year_Month', y='Default_Rate', hue='Region', data=monthly_defaults, marker='o')
plt.title('Monthly Default Rates Across Regions')
plt.xlabel('Year-Month')
plt.ylabel('Default Rate (%)')
plt.xticks(rotation=45)
plt.legend(title='Region')
plt.grid(True)
plt.show()


● Identify seasonal patterns in loan applications and disbursements.

In [ ]:
# Convert dates to datetime format
df['Application_Date'] = pd.to_datetime(df['Application_Date'], errors='coerce')
df['Disbursal_Date'] = pd.to_datetime(df['Disbursal_Date'], errors='coerce')

# Drop rows with missing dates (optional)
df = df.dropna(subset=['Application_Date', 'Disbursal_Date'])

# Create columns for Month and Year
df['Application_Month'] = df['Application_Date'].dt.month
df['Application_Year'] = df['Application_Date'].dt.year

df['Disbursal_Month'] = df['Disbursal_Date'].dt.month
df['Disbursal_Year'] = df['Disbursal_Date'].dt.year

# Grouping by Month
monthly_trends = df.groupby('Application_Month').agg(
    Total_Applications=('Application_ID', 'count'),
    Total_Disbursed=('Loan_ID_x', 'count')
).reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(x='Application_Month', y='Total_Applications', data=monthly_trends, marker='o', label="Applications")
sns.lineplot(x='Application_Month', y='Total_Disbursed', data=monthly_trends, marker='s', label="Disbursements")

plt.title("Seasonal Trends in Loan Applications and Disbursements")
plt.xlabel("Month")
plt.ylabel("Count")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
plt.legend()
plt.grid(True)
plt.show()


17. Customer Behavior Analysis 

● Categorize customers based on their repayment behavior (e.g., always on 
time, occasional defaulters, frequent defaulters). 

In [ ]:
# Group data by Customer_ID and calculate repayment behavior
customer_repayment = df.groupby('Customer_ID').agg(
    Total_Loans=('Loan_ID_x', 'count'),
    Total_Defaults=('Default_Flag', 'sum'),
    Total_Overdue_Amount=('Overdue_Amount', 'sum'),
    Total_On_Time_Payments=('On_Time_Payment', 'sum')
).reset_index()

# Function to categorize repayment behavior
def categorize_repayment(row):
    if row['Total_Defaults'] == 0:
        return 'Always On Time'
    elif row['Total_Defaults'] > 0 and row['Total_Defaults'] < 3:
        return 'Occasional Defaulter'
    else:
        return 'Frequent Defaulter'

# Apply categorization function
customer_repayment['Repayment_Behavior'] = customer_repayment.apply(categorize_repayment, axis=1)

plt.figure(figsize=(8, 5))
sns.countplot(x='Repayment_Behavior', data=customer_repayment, palette="coolwarm")

plt.title("Customer Segmentation Based on Repayment Behavior")
plt.xlabel("Repayment Behavior")
plt.ylabel("Number of Customers")
plt.show()


● Analyze patterns in loan approval and rejection reasons segmented by 
customer demographics. 

In [ ]:
# Grouping by customer demographics
approval_rejection_analysis = df.groupby(['Age', 'Annual_Income', 'Credit_Score', 'Employment_Status']).agg(
    Total_Applications=('Application_ID', 'count'),
    Approved_Applications=('Approval_Status', lambda x: (x == 'Approved').sum()),
    Rejected_Applications=('Approval_Status', lambda x: (x == 'Rejected').sum())
).reset_index()

# Calculate approval and rejection rates
approval_rejection_analysis['Approval_Rate'] = approval_rejection_analysis['Approved_Applications'] / approval_rejection_analysis['Total_Applications']
approval_rejection_analysis['Rejection_Rate'] = approval_rejection_analysis['Rejected_Applications'] / approval_rejection_analysis['Total_Applications']

df_rejected = df[df['Approval_Status'] == 'Rejected']
print(df_rejected.head())  # Check if the subset contains rejection reasons

# Replace 'Rejection_Reason' with the correct column name
correct_column_name = 'Reason_For_Rejection'  # Change this based on Step 1 results

print(df['Approval_Status'].unique())  # Check the unique values

df_rejected = df[df['Approval_Status'] == 'Rejected']
print(df_rejected.head())  # Preview rejected applications

df_rejected['Rejection_Reason'] = df_rejected.apply(lambda row: 
    "Low Credit Score" if row['Credit_Score'] < 600 else
    "High Debt-to-Income Ratio" if (row['EMI_Amount'] / row['Annual_Income']) > 0.4 else
    "Insufficient Income" if row['Annual_Income'] < row['Loan_Amount'] else
    "Other",
    axis=1
)
rejection_counts = df_rejected['Rejection_Reason'].value_counts()
print(rejection_counts)

● Identify high-value customers with consistent repayment histories. 

In [ ]:
# Define the threshold for high-value customers
high_loan_threshold = df['Loan_Amount'].quantile(0.75)  # Top 25% of loan amounts
high_credit_score = 750  # Good credit score threshold

# Filter customers who meet these conditions
high_value_customers = df[
    (df['Loan_Amount'] >= high_loan_threshold) &  # High loan amount
    (df['Credit_Score'] >= high_credit_score) &  # Good credit score
    (df['Default_Flag'] == 0)  # Never defaulted
]

# Display the top high-value customers
print(high_value_customers[['Customer_ID', 'Full_Name', 'Loan_Amount', 'Credit_Score']].head(10))

high_value_customers_sorted = high_value_customers.sort_values(by='Loan_Amount', ascending=False)
print(high_value_customers_sorted[['Customer_ID', 'Full_Name', 'Loan_Amount', 'Credit_Score']].head(10))

customer_segments = high_value_customers.groupby('Segment').size()
print(customer_segments)

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
sns.histplot(high_value_customers['Loan_Amount'], bins=20, kde=True)
plt.title('Distribution of Loan Amounts for High-Value Customers')
plt.show()




In [ ]:
df.columns

18. Risk Assessment 

● Develop a risk matrix for loan products based on Default_Amount, 
Loan_Term, and Interest_Rate. 

In [ ]:
import numpy as np

# Define risk categories for Default_Amount
def categorize_default_risk(amount):
    if amount < 50000:
        return "Low"
    elif 50000 <= amount <= 200000:
        return "Medium"
    else:
        return "High"

# Define risk categories for Loan_Term
def categorize_term_risk(term):
    if term < 24:
        return "Low"
    elif 24 <= term <= 60:
        return "Medium"
    else:
        return "High"

# Define risk categories for Interest_Rate
def categorize_interest_risk(rate):
    if rate < 8:
        return "Low"
    elif 8 <= rate <= 15:
        return "Medium"
    else:
        return "High"

# Apply functions to create new columns
df['Default_Risk'] = df['Default_Amount'].apply(categorize_default_risk)
df['Term_Risk'] = df['Loan_Term'].apply(categorize_term_risk)
df['Interest_Risk'] = df['Interest_Rate'].apply(categorize_interest_risk)

# Combine the three risk factors to create an overall risk category
df['Risk_Level'] = df[['Default_Risk', 'Term_Risk', 'Interest_Risk']].mode(axis=1)[0]

# Display a few rows to check the risk categories
df[['Loan_ID_y', 'Default_Amount', 'Loan_Term', 'Interest_Rate', 'Risk_Level']].head(10)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Pivot table for visualization
risk_matrix = df.pivot_table(index="Loan_Term", columns="Interest_Rate", values="Default_Amount", aggfunc='mean')

# Plot heatmap
plt.figure(figsize=(10,6))
sns.heatmap(risk_matrix, cmap="coolwarm", annot=True, fmt=".0f", linewidths=0.5)
plt.title("Risk Matrix - Default Amount by Loan Term & Interest Rate")
plt.xlabel("Interest Rate (%)")
plt.ylabel("Loan Term (months)")
plt.show()


● Rank loan types by risk level and suggest mitigation strategies. 

In [ ]:
# Group by Loan Purpose (or Loan_Type, if available)
loan_risk = df.groupby('Loan_Purpose').agg(
    Total_Loans=('Loan_ID_y', 'count'),
    Total_Defaults=('Default_Flag', 'sum'),
    Avg_Default_Amount=('Default_Amount', 'mean'),
    Avg_Loan_Term=('Loan_Term', 'mean'),
    Avg_Interest_Rate=('Interest_Rate', 'mean')
).reset_index()

# Calculate Default Rate
loan_risk['Default_Rate'] = loan_risk['Total_Defaults'] / loan_risk['Total_Loans']

# Rank Loan Types by Risk (Higher Default Rate & Amount = Higher Risk)
loan_risk['Risk_Score'] = (loan_risk['Default_Rate'] * 0.5) + \
                          (loan_risk['Avg_Default_Amount'] * 0.3) + \
                          (loan_risk['Avg_Interest_Rate'] * 0.2)

# Rank from highest risk to lowest
loan_risk = loan_risk.sort_values(by='Risk_Score', ascending=False)

# Display the top high-risk loan types
loan_risk[['Loan_Purpose', 'Default_Rate', 'Avg_Default_Amount', 'Risk_Score']].head(10)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
sns.barplot(y=loan_risk['Risk_Score'], x=loan_risk['Loan_Purpose'], palette="Reds_r")
plt.xlabel("Risk Score")
plt.ylabel("Loan Purpose")
plt.title("Loan Types Ranked by Risk Level")
plt.show()


● Analyze high-risk customer segments by credit score and income. 

In [ ]:
# Define Risk Categories Based on Credit Score
def credit_risk_category(score):
    if score < 600:
        return 'High Risk'
    elif 600 <= score < 750:
        return 'Medium Risk'
    else:
        return 'Low Risk'

# Apply Credit Risk Segmentation
df['Credit_Risk'] = df['Credit_Score'].apply(credit_risk_category)

# Define Income Segments
def income_category(income):
    if income < 500000:
        return 'Low Income'
    elif 500000 <= income <= 1500000:
        return 'Middle Income'
    else:
        return 'High Income'

# Apply Income Segmentation
df['Income_Category'] = df['Annual_Income'].apply(income_category)

# Count Defaults in Each Segment
risk_analysis = df.groupby(['Credit_Risk', 'Income_Category'])['Default_Flag'].sum().reset_index()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.barplot(x="Income_Category", y="Default_Flag", hue="Credit_Risk", data=risk_analysis, palette="Reds")
plt.xlabel("Income Category")
plt.ylabel("Number of Defaults")
plt.title("Default Count by Credit Score and Income Segments")
plt.legend(title="Credit Risk")
plt.show()


19. Time to Default Analysis 

● Calculate the average time from loan disbursement to default for overdue 
loans.

In [ ]:
import pandas as pd

# Ensure date columns are in datetime format
df['Disbursal_Date'] = pd.to_datetime(df['Disbursal_Date'])
df['Default_Date'] = pd.to_datetime(df['Default_Date'])

# Filter only defaulted loans
overdue_loans = df[df['Default_Flag'] == 1]

# Calculate time difference in days
overdue_loans['Days_to_Default'] = (overdue_loans['Default_Date'] - overdue_loans['Disbursal_Date']).dt.days

# Compute the average time to default
average_time_to_default = overdue_loans['Days_to_Default'].mean()

print(f"📌 The average time from loan disbursement to default is {average_time_to_default:.2f} days.")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
sns.histplot(overdue_loans['Days_to_Default'], bins=30, kde=True, color='red')
plt.xlabel("Days from Disbursement to Default")
plt.ylabel("Number of Loans")
plt.title("Distribution of Time to Default for Overdue Loans")
plt.show()


● Identify loan purposes with the shortest time to default. 

In [ ]:
import pandas as pd

# Ensure date columns are in datetime format
df['Disbursal_Date'] = pd.to_datetime(df['Disbursal_Date'])
df['Default_Date'] = pd.to_datetime(df['Default_Date'])

# Filter only defaulted loans
defaulted_loans = df[df['Default_Flag'] == 1]

# Calculate time difference in days
defaulted_loans['Days_to_Default'] = (defaulted_loans['Default_Date'] - defaulted_loans['Disbursal_Date']).dt.days

# Group by Loan_Purpose and calculate average time to default
loan_purpose_analysis = defaulted_loans.groupby('Loan_Purpose')['Days_to_Default'].mean().reset_index()

# Sort by shortest time to default
loan_purpose_analysis = loan_purpose_analysis.sort_values(by='Days_to_Default')

# Display the top 5 loan purposes with the shortest time to default
print("📌 Loan Purposes with the Shortest Time to Default:")
print(loan_purpose_analysis.head(5))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,6))
sns.barplot(x='Days_to_Default', y='Loan_Purpose', data=loan_purpose_analysis.head(10), palette="Reds_r")
plt.xlabel("Average Days to Default")
plt.ylabel("Loan Purpose")
plt.title("Loan Purposes with the Shortest Time to Default")
plt.show()


● Compare the time to default across customer demographics. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure date columns are in datetime format
df['Disbursal_Date'] = pd.to_datetime(df['Disbursal_Date'])
df['Default_Date'] = pd.to_datetime(df['Default_Date'])

# Filter only defaulted loans
defaulted_loans = df[df['Default_Flag'] == 1]

# Calculate time to default in days
defaulted_loans['Days_to_Default'] = (defaulted_loans['Default_Date'] - defaulted_loans['Disbursal_Date']).dt.days

# Group by customer demographics and calculate average time to default
age_analysis = defaulted_loans.groupby('Age')['Days_to_Default'].mean().reset_index()
income_analysis = defaulted_loans.groupby('Annual_Income')['Days_to_Default'].mean().reset_index()
credit_score_analysis = defaulted_loans.groupby('Credit_Score')['Days_to_Default'].mean().reset_index()
employment_analysis = defaulted_loans.groupby('Employment_Status')['Days_to_Default'].mean().reset_index()

# Display results
print("📌 Average Time to Default by Age Group:")
print(age_analysis.sort_values(by='Days_to_Default').head(5))

print("\n📌 Average Time to Default by Annual Income:")
print(income_analysis.sort_values(by='Days_to_Default').head(5))

print("\n📌 Average Time to Default by Credit Score:")
print(credit_score_analysis.sort_values(by='Days_to_Default').head(5))

print("\n📌 Average Time to Default by Employment Status:")
print(employment_analysis.sort_values(by='Days_to_Default').head(5))


In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(x='Credit_Score', y='Days_to_Default', data=credit_score_analysis, marker="o", color="red")
plt.xlabel("Credit Score")
plt.ylabel("Average Days to Default")
plt.title("Time to Default by Credit Score")
plt.show()


20. Transaction Pattern Analysis 

● Identify customers with irregular repayment patterns. 

In [ ]:
import pandas as pd

# Filter required columns
df_repayment = df[['Customer_ID', 'Loan_ID_x', 'EMI_Amount', 'Overdue_Amount', 'On_Time_Payment']]

# Identify late payers (customers with overdue amounts)
df_late_payers = df_repayment[df_repayment['Overdue_Amount'] > 0]

# Group by customer and count overdue occurrences
customer_irregular_repayments = df_late_payers.groupby('Customer_ID').agg({
    'Loan_ID_x': 'count',  # Number of loans with overdue
    'Overdue_Amount': 'sum',  # Total overdue amount
    'On_Time_Payment': 'mean'  # Percentage of on-time payments
}).reset_index()

# Rename columns
customer_irregular_repayments.rename(columns={'Loan_ID_x': 'Overdue_Loan_Count'}, inplace=True)

# Classify customers based on repayment behavior
def classify_irregular_repayment(row):
    if row['On_Time_Payment'] < 0.5:  # Less than 50% on-time payments
        return 'Frequent Defaulter'
    elif row['Overdue_Loan_Count'] > 2 and row['Overdue_Amount'] > 50000:
        return 'High-Risk Borrower'
    elif row['Overdue_Loan_Count'] > 1:
        return 'Occasional Defaulter'
    else:
        return 'Moderate Risk'

customer_irregular_repayments['Repayment_Category'] = customer_irregular_repayments.apply(classify_irregular_repayment, axis=1)

# Display top irregular repayment customers
print(customer_irregular_repayments.head(10))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
sns.countplot(x='Repayment_Category', data=customer_irregular_repayments, palette="coolwarm")
plt.xlabel("Repayment Category")
plt.ylabel("Number of Customers")
plt.title("Classification of Customers by Repayment Behavior")
plt.xticks(rotation=45)
plt.show()


● Analyze penalty payments as a proportion of total transactions. 

In [ ]:
import pandas as pd

# Ensure required columns exist
if 'Overdue_Fee' in df.columns and 'Amount' in df.columns:
    # Calculate total penalty payments and total transaction amount
    total_penalty_payments = df['Overdue_Fee'].sum()
    total_transaction_amount = df['Amount'].sum()

    # Calculate the proportion of penalty payments
    penalty_proportion = (total_penalty_payments / total_transaction_amount) * 100

    print(f"Total Penalty Payments: {total_penalty_payments}")
    print(f"Total Transaction Amount: {total_transaction_amount}")
    print(f"Penalty Payment Proportion: {penalty_proportion:.2f}%")
else:
    print("Required columns ('Overdue_Fee' and 'Amount') not found in the dataset.")


In [ ]:
import matplotlib.pyplot as plt

labels = ['Penalty Payments', 'Regular Payments']
sizes = [total_penalty_payments, total_transaction_amount - total_penalty_payments]
colors = ['red', 'green']
explode = (0.1, 0)  # Slightly separate the penalty payments section

plt.figure(figsize=(7, 7))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, explode=explode, shadow=True, startangle=140)
plt.title("Proportion of Penalty Payments in Total Transactions")
plt.show()


● Compare transaction amounts for overdue vs. non-overdue loans. 

In [ ]:
import pandas as pd

# Ensure required columns exist
if 'Overdue_Amount' in df.columns and 'Amount' in df.columns:
    # Categorize loans as overdue or non-overdue
    overdue_loans = df[df['Overdue_Amount'] > 0]
    non_overdue_loans = df[df['Overdue_Amount'] == 0]

    # Calculate total and average transaction amounts
    overdue_total = overdue_loans['Amount'].sum()
    non_overdue_total = non_overdue_loans['Amount'].sum()

    overdue_avg = overdue_loans['Amount'].mean()
    non_overdue_avg = non_overdue_loans['Amount'].mean()

    # Print results
    print(f"Total Transaction Amount for Overdue Loans: {overdue_total}")
    print(f"Total Transaction Amount for Non-Overdue Loans: {non_overdue_total}")
    print(f"Average Transaction Amount for Overdue Loans: {overdue_avg:.2f}")
    print(f"Average Transaction Amount for Non-Overdue Loans: {non_overdue_avg:.2f}")
else:
    print("Required columns ('Overdue_Amount' and 'Amount') not found in the dataset.")


In [ ]:
import matplotlib.pyplot as plt

categories = ['Overdue Loans', 'Non-Overdue Loans']
totals = [overdue_total, non_overdue_total]
averages = [overdue_avg, non_overdue_avg]

plt.figure(figsize=(10,5))

# Plot total transaction amounts
plt.subplot(1,2,1)
plt.bar(categories, totals, color=['red', 'green'])
plt.title("Total Transaction Amount")
plt.ylabel("Amount")

# Plot average transaction amounts
plt.subplot(1,2,2)
plt.bar(categories, averages, color=['red', 'green'])
plt.title("Average Transaction Amount")
plt.ylabel("Amount")

plt.tight_layout()
plt.show()


Collective Agenda 

The collective agenda is to provide Hero FinCorp with actionable 
recommendations to: 

1. Minimize loan defaults by identifying high-risk customers and regions. 

In [ ]:
# Filter high-risk customers based on predefined thresholds
high_risk_customers = df[(df['Credit_Score'] < 600) | (df['Default_Flag'] == 1)]

# Display summary statistics for high-risk customers
print("High-Risk Customers Summary:")
print(high_risk_customers[['Customer_ID', 'Credit_Score', 'Annual_Income', 'Default_Flag']].describe())

# Count of high-risk customers
num_high_risk = high_risk_customers.shape[0]
print(f"Total High-Risk Customers: {num_high_risk}")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
sns.histplot(high_risk_customers['Credit_Score'], bins=20, color="red", kde=True)
plt.title("Distribution of Credit Scores Among High-Risk Customers")
plt.xlabel("Credit Score")
plt.ylabel("Count")
plt.show()


2. Optimize branch operations by improving processing time and recovery 
rates. 

In [ ]:
# Calculate processing time
df['Processing_Time'] = (pd.to_datetime(df['Approval_Date']) - pd.to_datetime(df['Application_Date'])).dt.days

# Group by branch and calculate average processing time
branch_processing = df.groupby('Branch_ID')['Processing_Time'].mean().reset_index()

# Display branches with the highest processing time
print("Branches with Highest Processing Time:")
print(branch_processing.sort_values(by='Processing_Time', ascending=False).head(10))


In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=branch_processing['Branch_ID'], y=branch_processing['Processing_Time'], palette="Blues_r")
plt.xticks(rotation=45)
plt.title("Average Loan Processing Time by Branch")
plt.ylabel("Processing Time (Days)")
plt.show()


In [ ]:
# Group by branch and calculate recovery rate
branch_recovery = df.groupby('Branch_ID').agg(
    Total_Default_Amount=('Default_Amount', 'sum'),
    Total_Recovery_Amount=('Recovery_Amount', 'sum')
)

# Calculate recovery rate
branch_recovery['Recovery_Rate'] = (branch_recovery['Total_Recovery_Amount'] / branch_recovery['Total_Default_Amount']) * 100

# Display top branches with the highest recovery rate
print("Top Branches by Recovery Rate:")
print(branch_recovery.sort_values(by='Recovery_Rate', ascending=False).head(10))


In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=branch_recovery.index, y=branch_recovery['Recovery_Rate'], palette="Greens_r")
plt.xticks(rotation=45)
plt.title("Branch-Wise Recovery Rates")
plt.ylabel("Recovery Rate (%)")
plt.show()


In [ ]:
# Merge branch processing and recovery data
branch_efficiency = branch_processing.merge(branch_recovery, on='Branch_ID')

# Display branch efficiency summary
print("Branch Efficiency Summary:")
print(branch_efficiency.head(10))


In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(data=branch_efficiency, x='Processing_Time', y='Recovery_Rate', hue='Branch_ID', palette="coolwarm")
plt.title("Branch Efficiency: Processing Time vs. Recovery Rate")
plt.xlabel("Average Processing Time (Days)")
plt.ylabel("Recovery Rate (%)")
plt.show()


3. Enhance profitability through better customer segmentation and interest 
income strategies

In [ ]:
# Define customer segments based on income
def income_segment(income):
    if income < 300000:
        return 'Low Income'
    elif income < 800000:
        return 'Middle Income'
    else:
        return 'High Income'

df['Income_Segment'] = df['Annual_Income'].apply(income_segment)

# Define risk levels based on Credit Score
def credit_segment(score):
    if score < 600:
        return 'High Risk'
    elif score < 750:
        return 'Medium Risk'
    else:
        return 'Low Risk'

df['Credit_Segment'] = df['Credit_Score'].apply(credit_segment)

# Group by segments and analyze default trends
customer_segments = df.groupby(['Income_Segment', 'Credit_Segment']).agg(
    Total_Loans=('Loan_Amount', 'count'),
    Avg_Loan_Amount=('Loan_Amount', 'mean'),
    Default_Rate=('Default_Flag', 'mean')
).reset_index()

# Display segment-wise analysis
print(customer_segments)


In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=customer_segments, x="Income_Segment", y="Default_Rate", hue="Credit_Segment", palette="coolwarm")
plt.title("Default Rates by Customer Segments")
plt.ylabel("Default Rate (%)")
plt.show()


In [ ]:
# Calculate interest income
df['Interest_Income'] = (df['Loan_Amount'] * df['Interest_Rate'] * df['Loan_Term']) / 100

# Aggregate interest income by loan purpose
interest_by_purpose = df.groupby('Loan_Purpose').agg(
    Total_Interest_Income=('Interest_Income', 'sum'),
    Avg_Interest_Rate=('Interest_Rate', 'mean')
).reset_index()

# Display most profitable loan purposes
print(interest_by_purpose.sort_values(by='Total_Interest_Income', ascending=False).head(10))


In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=interest_by_purpose, x="Loan_Purpose", y="Total_Interest_Income", palette="viridis")
plt.xticks(rotation=45)
plt.title("Total Interest Income by Loan Purpose")
plt.ylabel("Total Interest Income (in ₹)")
plt.show()


In [ ]:
# Group by branch and calculate total interest income
branch_profitability = df.groupby('Branch_ID').agg(
    Total_Interest_Income=('Interest_Income', 'sum'),
    Avg_Loan_Amount=('Loan_Amount', 'mean'),
    Total_Customers=('Customer_ID', 'count')
).reset_index()

# Display top branches by interest income
print(branch_profitability.sort_values(by='Total_Interest_Income', ascending=False).head(10))


In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=branch_profitability, x="Branch_ID", y="Total_Interest_Income", palette="coolwarm")
plt.xticks(rotation=45)
plt.title("Branch-Wise Interest Income")
plt.ylabel("Total Interest Income (in ₹)")
plt.show()
